In [1]:
import gspread

In [4]:
gc = gspread.oauth(credentials_filename='credentials.json')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=516615602945-71tj4rma0e27fk3700s5eq87564pkoiv.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54347%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=AQIDu8Xz4wrB48vMP8HKxRVu3WGKa9&access_type=offline


In [5]:
sht2 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1xEqT4kghsaAZYTdN7SN9UHN31Ehx29OykbG1Mjpm-go/edit?gid=0#gid=0')

sht2

<Spreadsheet 'Copy of Torch Arrival Data' id:1xEqT4kghsaAZYTdN7SN9UHN31Ehx29OykbG1Mjpm-go>

In [7]:
worksheet = sht2.get_worksheet(0)
worksheet = sht2.worksheet("Shopee")
worksheet

<Worksheet 'Shopee' id:0>

In [8]:
sht2.worksheets()

[<Worksheet 'Shopee' id:0>,
 <Worksheet 'Tokopedia' id:715003248>,
 <Worksheet 'TokoMargonda' id:1960044452>]

In [9]:
import pandas as pd

dataframe = pd.DataFrame(worksheet.get_all_records())